In [0]:
import mlflow

guidelines = {
    "english": ["The response must be in English"],
    "clarity": ["The response must be clear, coherent, and concise"],
}
eval_set = [
    {
        "request": "What is the capital of England?",
        "response": "The capital of England is London.",
    },
    {
        "request": "What is the capital of France?",
        "response": "La capitale de la France est Paris.",
    },
    {
        "request": "What is the capital of the USA?",
        "response": "The capital of the United States of America, commonly referred to as the USA, is Washington, D.C. This city, which is not part of any state and is instead a federal district, was founded on July 16, 1790. It is named after George Washington, the first President of the United States, and Christopher Columbus, the explorer after whom the Americas are named. Washington, D.C. serves as the political hub of the country, housing the three branches of the federal government: the Executive, led by the President; the Legislative, consisting of the Congress; and the Judicial, headed by the Supreme Court. Additionally, it is home to numerous national monuments and museums, primarily situated on or around the National Mall. The city plays a pivotal role in the nation's history and governance, making it a significant location not only for its political importance but also for its cultural and historical heritage.",
    },
]

mlflow.evaluate(
    data=eval_set,
    model_type="databricks-agent",
    evaluator_config={
        "databricks-agent": {
            "metrics": ["guideline_adherence"],
            "global_guidelines": guidelines,
        }
    },
)

In [0]:
import mlflow
from databricks.agents.evals import metric
from mlflow.evaluation import Assessment

eval_set = [
    {
        "request": "What is the capital of England?",
        "response": "The capital of England is London.",
    },
    {
        "request": "What is the capital of France?",
        "response": "",
    },
    
]

# Example custom metric
@metric
def not_empty(response):
    # "yes" for Pass and "no" for Fail.
    score = "yes" if response and response.strip() != "" else "no"
    rationale = "The response is empty" if score == "no" else "The response is not empty"
    return Assessment(name="not_empty", value=score, rationale=rationale)

# Call the metric with mlflow.evaluate(...)
with mlflow.start_run(run_name="example_run"):
    eval_results = mlflow.evaluate(
        data=eval_set,
        model_type="databricks-agent",
        extra_metrics=[not_empty],
    )